In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

/tmp/ipykernel_3917/2632389468.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
embedder = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

In [4]:
# Import necessary libraries
from langchain_chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.schema import Document



# Initialize ChromaDB vector store
vector_store = Chroma(embedding_function=embedder)

# Create some sample documents
documents = [
    Document(page_content="This is the first document.", metadata={"label": "doc1"}),
    Document(page_content="This is the second document.", metadata={"label": "doc2"}),
    Document(page_content="This is the third document.", metadata={"label": "doc3"}),
]

# Add documents to the vector store
vector_store.add_documents(documents)

# Perform a similarity search
query = "Find documents similar to this query."
results = vector_store.similarity_search(query, k=2)

# Print the search results
print("Search Results:")
for result in results:
    print(f"Content: {result.page_content}, Metadata: {result.metadata}")


/home/onyxia/work/ollama-langchain-mistral-notebook/.env/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


Search Results:
Content: This is the second document., Metadata: {'label': 'doc2'}
Content: This is the first document., Metadata: {'label': 'doc1'}
